# Setup

In [1]:
!python -V

Python 3.10.12


In [2]:
# %load_ext sql

In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2023/07/07 15:34:52 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/07 15:34:52 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/mnt/nas/Drive/Dev/zoomcamp/mlops/02-experiment-tracking/mlruns/1', creation_time=1688706055743, experiment_id='1', last_update_time=1688706055743, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [13]:
# %sql sqlite:///mlflow.db
# %sql CREATE TABLE IF NOT EXISTS experiments (experiment_id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, artifact_location TEXT, lifecycle_stage TEXT)

## Load Data

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    # df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    # df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2022-02.parquet')

In [9]:
len(df_train), len(df_val)

(59603, 66097)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# Linear Regression

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.928931234129946

In [14]:
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "william")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2022-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

# XGBoost

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3), exp(0) - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[13:47:02] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.64681                          
[1]	validation-rmse:15.89727                          
[2]	validation-rmse:15.19493                          
[3]	validation-rmse:14.53621                          
[4]	validation-rmse:13.91993                          
[5]	validation-rmse:13.34186                          
[6]	validation-rmse:12.80228                          
[7]	validation-rmse:12.29786                          
[8]	validation-rmse:11.82749                          
[9]	validation-rmse:11.38677                          
[10]	validation-rmse:10.97792                         
[11]	validation-rmse:10.59647                         
[12]	validation-rmse:10.24265                         
[13]	validation-rmse:9.91267                          
[14]	validation-rmse:9.60683                          
[15]	validation-rmse:9.32299                          
[

In [18]:
mlflow.xgboost.autolog(disable=True)

In [22]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.0777668648586099,
        'max_depth': 14,
        'min_child_weight': 1.1427124390567596,
        'objective': 'reg:linear',
        'reg_alpha': 0.014606162719210855,
        'reg_lambda': 0.07473798860264463,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[15:04:08] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.31996
[1]	validation-rmse:15.29183
[2]	validation-rmse:14.35497
[3]	validation-rmse:13.50357
[4]	validation-rmse:12.72908
[5]	validation-rmse:12.02618
[6]	validation-rmse:11.38796
[7]	validation-rmse:10.81226
[8]	validation-rmse:10.29205
[9]	validation-rmse:9.82598
[10]	validation-rmse:9.40558
[11]	validation-rmse:9.02905
[12]	validation-rmse:8.69139
[13]	validation-rmse:8.39093
[14]	validation-rmse:8.12206
[15]	validation-rmse:7.88273
[16]	validation-rmse:7.66957
[17]	validation-rmse:7.48232
[18]	validation-rmse:7.31390
[19]	validation-rmse:7.16520
[20]	validation-rmse:7.03247
[21]	validation-rmse:6.91595
[22]	validation-rmse:6.81295
[23]	validation-rmse:6.72240
[24]	validation-rmse:6.64034
[25]	validation-rmse:6.56853
[26]	validation-rmse:6.50409
[27]	validation-rmse:6.44642
[28]	validation-rmse:6.39614
[29]	validation-rmse:6.35236
[30]	validat

In [24]:
logged_model = 'runs:/56c34140806b44698077835bcc30c86a/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

2023/07/07 15:10:13 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[15:10:13] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 56c34140806b44698077835bcc30c86a

In [34]:
y_pred = loaded_model.predict(X_val)

y_pred[:10]

array([ 6.7548985,  4.262297 , 25.351759 , 35.717896 , 27.083    ,
        9.599475 , 21.359627 ,  4.1045218, 15.539541 ,  5.542191 ],
      dtype=float32)

In [25]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

xgboost_model

[15:11:17] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [26]:
y_pred = xgboost_model.predict(valid)

y_pred[:10]

array([ 6.7548985,  4.262297 , 25.351759 , 35.717896 , 27.083    ,
        9.599475 , 21.359627 ,  4.1045218, 15.539541 ,  5.542191 ],
      dtype=float32)

# Scikit-learn

In [13]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2022-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2022-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2023/07/07 15:35:20 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/07/07 15:38:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/william/.pyenv/versions/3.10.12/envs/zoomcamp-exp-track/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/07/07 15:38:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/07/07 15:38:58 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/07/07 15:47:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/william/.pyenv/versions/3.10.12/envs/zoomcamp-exp-track/lib